In [35]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
%run $header_path

C:\Users\raphael.wargon\Documents\GitHub\sorting_univ\script/headers/
default
initalizing spark nlp session w gpu


In [63]:
df_inst = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'institutions.parquet')
           .withColumn('max_updated_date', func.max(func.col('updated_date')).over(Window.partitionBy('id')))
           .filter(func.col('updated_date')==func.col('max_updated_date'))
          )

# Principal higher ed institutions

In [4]:
fr_esr = spark.read.format('csv').option("delimiter", ";").option("header","true").load('file:\\' + scanR_path + '\\fr-esr-principaux-etablissements-enseignement-superieur.csv')

In [5]:
fr_esr = (fr_esr
          .select(func.col('libellé').alias('name'), func.col('nom court').alias('short_name'),
                  func.col('identifiant_idref').alias('idref'), func.col('Identifiant ROR').alias('ror'),
                  func.col('Identifiant wikidata').alias('wikidata'), func.col("type d'établissement").alias('type'),
                 func.col("secteur d'établissement").alias('secteur'))
          .distinct()
         )
fr_esr.show()
          

+--------------------+--------------------+---------+---------+---------+-------------------+-------+
|                name|          short_name|    idref|      ror| wikidata|               type|secteur|
+--------------------+--------------------+---------+---------+---------+-------------------+-------+
|Université de Rennes|          UNIVRENNES|26693823X|015m7wh34| Q1987282|         Université| public|
|Institut français...|                IFAO|026387395|055pd2p26| Q1472454|Autre établissement| public|
|Université Paris ...|             UP Cité|236453505|05f82e368|Q55849612|         Université| public|
|École des hautes ...|           HEC Paris|027321118|0423jsj19|  Q273535|              École|  privé|
|Institut national...|        INSA de Lyon|052444724|050jn9y42| Q1633859|              École| public|
|Université de Poi...| Université Poitiers|026403765|04xhy8q59|  Q661056|         Université| public|
|Neoma Business Sc...|            Neoma BS|250789329|01g1pe685|Q16665962|         

In [6]:
merged_by_ror = (df_inst
                 .select('id', 'ror')
                 .join(fr_esr
                       .withColumn('ror', func.concat(func.lit('https://ror.org/'), func.col('ror'))), on = 'ror', how ='inner')
                 .select(func.regexp_replace(func.col('id'), 'https://openalex.org/', '').alias('inst_id'),'idref','name','ror','type',"secteur")
                 .distinct()
                )
merged_by_ror.count()

215

In [7]:
merged_by_ror.show()

+-----------+---------+--------------------+--------------------+-------------------+-------+
|    inst_id|    idref|                name|                 ror|               type|secteur|
+-----------+---------+--------------------+--------------------+-------------------+-------+
| I187986737|026370921|   Collège de France|https://ror.org/0...|Grand établissement| public|
| I138816502|029473284|Université de Cor...|https://ror.org/0...|         Université| public|
|I2802487185|030969379|Université du Lit...|https://ror.org/0...|         Université| public|
|I2746051580|241597595|Université Paris ...|https://ror.org/0...|Grand établissement| public|
| I110017253|026404478| Université de Tours|https://ror.org/0...|         Université| public|
|I4210145102|238327159|Institut polytech...|https://ror.org/0...|              École| public|
|  I40434647|026403587|Université Paris ...|https://ror.org/0...|         Université| public|
| I277688954|241345251|Université Paris-...|https://ror.org/

In [8]:
merged_by_ror.groupBy('type').count().show()

+-------------------+-----+
|               type|count|
+-------------------+-----+
|         Université|   61|
|Grand établissement|   27|
|              École|  117|
|Autre établissement|   10|
+-------------------+-----+



In [9]:
df_inst.filter(func.col('id').contains("I4403386658")).show()

+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+-------------+---------------+-------+------------+-----+---+-------------+-----------+------+----+-------+-------+------------+-------------+-----------+----------+
|associated_institutions|cited_by_count|country_code|country_id|counts_by_year|created_date|display_name|display_name_acronyms|display_name_alternatives|geo|homepage_url| id|ids|image_thumbnail_url|image_url|international|is_super_system|lineage|repositories|roles|ror|summary_stats|topic_share|topics|type|type_id|updated|updated_date|works_api_url|works_count|x_concepts|
+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+-------------+---------------+-------+------

In [11]:
merge = (inst.withColumn('to_join',
                                           func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",'')
                )
      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
.join(fr_esr
      .withColumn('to_join', 
                  func.explode(func.array([func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",''), 
                                           func.regexp_replace(func.lower(func.col('short_name')), " de| la| d'| l'| des| du| le| les",'')])))
      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
      , on = 'to_join', how ='inner'))
merge.count()

126

In [12]:
merge.show()

+--------------------+-----------+--------------------+---------+--------------+--------------------+--------------------+--------------------+-----+-------+----+--------------------+------------------+---------+---------+----------+-------------------+-------+
|             to_join|    inst_id|                name|     type|          city|         topic_share|          main_topic|              parent|fused|uni_pub|cnrs|                name|        short_name|    idref|      ror|  wikidata|               type|secteur|
+--------------------+-----------+--------------------+---------+--------------+--------------------+--------------------+--------------------+-----+-------+----+--------------------+------------------+---------+---------+----------+-------------------+-------+
|ecole nationale s...| I194210350|École Nationale S...|education|          Caen|[{Materials Scien...|   Materials Science|[{I194210350, Éco...|    0|      0|   0|École nationale s...|         ENSI Caen|190907991|01

In [13]:
inst.filter(func.col('inst_id')=="I4403386658").show()

+-------+----+----+----+-----------+----------+------+-----+-------+----+
|inst_id|name|type|city|topic_share|main_topic|parent|fused|uni_pub|cnrs|
+-------+----+----+----+-----------+----------+------+-----+-------+----+
+-------+----+----+----+-----------+----------+------+-----+-------+----+



In [16]:
fr_esr.count()

246

In [18]:
sorted_by_hand = { #'IAE Paris - Sorbonne Business School':
                   'Université de Mayotte':"I4403928291",
               "École d'ingénieurs des sciences aérospatiales" :"I58581550",
    'Université de technologie Tarbes Occitanie Pyrénées':"I119865316",
    #"Institut régional du travail social de Champagne-Ardennes"
    "Université Bourgogne Europe":"I177064439",
    "Université de Montpellier Paul-Valéry":'I208215962',
    #Institut national du service public
    "Nîmes université":"I66456633",
    #Y SCHOOLS
    "École nationale supérieure de paysage":"I4387154791",
    #Brest Business School
    #École supérieure de fonderie et de forge
    #Institut national des sciences et techniques nucléaires
    #Université Marie et Louis Pasteur
    "Université de Toulouse": "I17866349",
    #Institut supérieur du commerce de Paris,
    #École spéciale d'architecture
    #Institut supérieur de l'électronique et du numérique de Brest
    #Centre national des arts du cirque
    #Institut des relations publiques et de la communication d’Angers
    #ESIEE-IT
    #Institut français de la mode 
    #École nationale supérieure des arts et techniques du théâtre
    "Institut supérieur du bâtiment et des travaux publics": 'I4210160645',
    #Centre des hautes études de la construction
    #École supérieure d'art et de design d'Amiens
    "Université Jean Monnet":"I86767153",
    #Centre de ressources, d'expertise et de performance sportives de Toulouse 
    "École nationale supérieure du pétrole et des moteurs":"I4403386658",
    "Institut protestant de théologie":"I4210121454",
    "Institut supérieur international du parfum, de la cosmétique et de l'aromatique alimentaire":"I4387154221",
    "ICN Business School": "I4400600934",
    "École nationale supérieure des métiers de l'image et du son":'I4387156185',
    #Institut d'ingénierie informatique de Limoges
    "École de management Léonard De Vinci":"I4210121459"
    
                }

In [19]:
print([test[1]for test in sorted_by_hand.items()])

['I4403928291', 'I58581550', 'I119865316', 'I177064439', 'I208215962', 'I66456633', 'I4387154791', 'I17866349', 'I4210160645', 'I86767153', 'I4403386658', 'I4210121454', 'I4387154221', 'I4400600934', 'I4387156185', 'I4210121459']


In [20]:
expr_merge_hand = 'CASE WHEN ' + ' WHEN '.join(['name = "' + el[0] + '" THEN "' + el[1] + '"' for el in sorted_by_hand.items()] )+ ' END'
expr_merge_hand                  

'CASE WHEN name = "Université de Mayotte" THEN "I4403928291" WHEN name = "École d\'ingénieurs des sciences aérospatiales" THEN "I58581550" WHEN name = "Université de technologie Tarbes Occitanie Pyrénées" THEN "I119865316" WHEN name = "Université Bourgogne Europe" THEN "I177064439" WHEN name = "Université de Montpellier Paul-Valéry" THEN "I208215962" WHEN name = "Nîmes université" THEN "I66456633" WHEN name = "École nationale supérieure de paysage" THEN "I4387154791" WHEN name = "Université de Toulouse" THEN "I17866349" WHEN name = "Institut supérieur du bâtiment et des travaux publics" THEN "I4210160645" WHEN name = "Université Jean Monnet" THEN "I86767153" WHEN name = "École nationale supérieure du pétrole et des moteurs" THEN "I4403386658" WHEN name = "Institut protestant de théologie" THEN "I4210121454" WHEN name = "Institut supérieur international du parfum, de la cosmétique et de l\'aromatique alimentaire" THEN "I4387154221" WHEN name = "ICN Business School" THEN "I4400600934" WH

In [22]:
merged_by_hand = (fr_esr
                  .withColumn('inst_id', func.expr(expr_merge_hand))
                  .filter(func.col('inst_id').isNotNull())
                 )
merged_by_hand.show()

+--------------------+----------+---------+---------+----------+----------+-------+-----------+
|                name|short_name|    idref|      ror|  wikidata|      type|secteur|    inst_id|
+--------------------+----------+---------+---------+----------+----------+-------+-----------+
|Université de May...|      NULL|262921715|02t0k0832|  Q2946102|Université| public|I4403928291|
|École d'ingénieur...|     ELISA|     NULL|     NULL| Q65162999|     École|  privé|  I58581550|
|Université de tec...|      NULL|     NULL|     NULL|Q124412825|     École| public| I119865316|
|Université Bourgo...|      NULL|     NULL|     NULL|      NULL|Université| public| I177064439|
|Université de Mon...|      NULL|     NULL|     NULL|      NULL|Université| public| I208215962|
|    Nîmes université|      NULL|     NULL|     NULL|      NULL|Université| public|  I66456633|
|École nationale s...|      ENSP|035622261|     NULL|  Q3578260|     École| public|I4387154791|
|Université de Tou...|      NULL|     NU

In [23]:
merged = (merged_by_ror
          .select('inst_id', 'idref', 'name', 'type','secteur')
          .unionAll(merged_by_hand.select('inst_id', 'idref','name', 'type','secteur'))
          .distinct()
         )
merged.show()

+-----------+-------------------+--------------------+-------------------+-------+
|    inst_id|              idref|                name|               type|secteur|
+-----------+-------------------+--------------------+-------------------+-------+
|  I51101395|          027361802|Université Paris ...|         Université| public|
|  I90183372|          157040569|Université de Lor...|Grand établissement| public|
| I277688954|          241345251|Université Paris-...|         Université| public|
|  I68947357|          131056549|Université de Str...|         Université| public|
|I4210126056|086908480,193857561|Institut national...|Autre établissement| public|
|I4210103002|          252404955|Université Clermo...|         Université| public|
|  I24715933|          026404451|Université de La ...|         Université| public|
| I161463649|          028222814|Audencia Business...|              École|  privé|
|I4210123735|          028888537|    Sciences Po Lyon|Autre établissement| public|
|I43

In [24]:
merged.count()

231

In [74]:
idex = {"idex_aix_marseille": #https://anr.fr/ProjetIA-11-IDEX-0001
        ['I21491767',"I4210142724",'I4210087487','I4387155629', # Aix-Marseille Université, Ecole centrale de Marseille, CHRU Marseille, IEP Aix 
                             # "I4210166444", "I4387153840","I4210128565"  #IRD Marseille, Inserm Paris, CEA Saclay
        ]
,
        'idex_paris_saclay':#https://anr.fr/ProjetIA-11-IDEX-0003
        ['I195731000',"I22248866", "I83860666", "I201181511","I11559806","I142476485","I4210091746","I205703379", # UVSQ, AgroParisTech, HEC,ENSTA, ENS Saclay, X, SupOptique, Télécom,
         "I84009706", "I4210088668", "I2801658355" ,"I2801148539","I4210107720","I88467170", "I4210126360"#ENSAE,INRAE, ONERA, Synchotron Soleil, Centrale Supélec, Univ Evry Val d'Essonne, Inria Saclay
         #,"I4387153840", "I4210128565"# Inserm paris,CEA Saclay
        ] 
         # missing : IHES,Systematic,Institut Villebon Georges Charpak, ADEPRINA
,      'idex_bordeaux' : #https://anr.fr/ProjetIA-10-IDEX-0003
        ['I3130945570',"I4210160189",'I139506611','I4210091158' #Univ Bordeaux Montaigne, Polytechnique Bordeaux, Sciences Po Bordeaux, ENSA Bordeaux
         #,"I4387153840", "I4210128565"# Inserm paris,CEA Saclay
        ]
,      "idex_psl": #https://anr.fr/ProjetIA-10-IDEX-0001
        ["I2746051580", #PSL
            'I187986737',"I149758196","I29607241",'I98910050',"I80043",'I2800004676' #CdF, Chimie ParisTech, ENS Ulm, ESPCI, Institut Curie, Observatoire de Paris
         , 'I56435720',"I4210166784","I4210107334","I4210118986","I4210126561" #Dauphine, Ecole Nat Sup Arts Deco, Beaux Arts Paris, conservatoire musique et danse, institut louis bachelier
         ,"I70768539", "I159885104","I4210127304","I4387156185", "I90669466","I4210092825","I4210100316" #Mines Paris, EPHE, fondation Rotschild, fémis, EHESS, Ecole Française d'Extrême Orient, Chartes, 
      #  , "I4210126360"#Inria Saclay
        ]
,      "idex_paris_cite": #https://anr.fr/ProjetIA-18-IDEX-0001
        ['I204730241',"I148261361","I4210091279","I205092303","I162346809","I1325388108",'I182627622',"I1326498283" #Paris Cité, institut de physique du globe, Sorbonne Paris-Nord, Sciences Po, INALCO, INED, Sorbonne Nouvelle, INRIA
         #,"I4210166444","I4387153840","I4210128565" # IRD Marseille, Inserm Paris, CEA Saclay
        ]
,      "idex_sorbonne_univ": #https://anr.fr/ProjetIA-11-IDEX-0004
        ["I39804081" ,"I4210123595","I102272798","I102516824", #Sorbonne Université, Museum d'histoire naturelle, UTC
         #,"I4210166444","I4387153840","I4210128565" # IRD Marseille, Inserm Paris, CEA Saclay
        ]
,      "idex_strasbourg": #https://anr.fr/ProjetIA-10-IDEX-0002
        ["I68947357" #Univ_strasbourg
                  #,"I4387153840" # Inserm paris
        ]
,      "idex_2_cote_azur": #https://anr.fr/ProjetIA-15-IDEX-0001
        [ 'I201841394','I4210147253', "I4210126779","I3018988418", "I4210111578", #UCA, CNRS côte d'azur, observatoire côte d'azur, CHU Nice, INSERM côte d'azur
         "I1902872","I90326151","I181418319" # Eurecom, EDHEC, SKEMA, 
         #,"I70768539" Mines Paris                            
         # "I4210166444"  #IRD Marseille # INRAE
        ]
,     'idex_2_grenoble': #https://anr.fr/ProjetIA-15-IDEX-0002
        ['I106785703',"I4210113225","I3020098449", "I4210150872", #Polytechnique Grenoble, ENSAG, CEA Grenoble, INSERM Alpes
         "I899635006","I2800555055","I4210101348","I4387155711","I4387154183" #UGA, CHU Grenoble, INRIA Grenoble, Sciences Po Grenoble, IRSTEA

         # "I4210155509" CNRS Alpes
        ]

,     'idex_annulee_toulouse': #https://anr.fr/ProjetIA-11-IDEX-0002
        ["I4210126056",'I17866349','I4210152422','I3131550300', #Centre Champollion, Univ Toulouse, Toulouse Jaurès,Toulouse Capitole,
         'I134560555',"I205747304","I196454796","I193033237" ,"I4210147367" # Toulouse Sabatier, Polytechnique Toulouse, INSA Toulouse, ISEA-SUPAéro, INRAE Occitanie
         , "I4210106734", "I4210145734","I4210092408","I176063091" # CNRS Occitanie, INSERM Occitanie, TSE, Ecole Vétérinaire Toulouse,
        ,"I4210091876","I119865316","I4210112425","I129162050" #Mines d'Albi, Université de Technologie de Tarbes, Sciences Po Toulouse,Toulouse Business School
         ,"I100296615",  "I100296615" #ENAC, Ecole nationale d'Ingénieurs de Purpan
         #,"I4210166444" #IRD Marseille
        ]
,     "idex_annulee_lyon":#https://anr.fr/ProjetIA-16-IDEX-0005
        ["I112936343","I203339264", "I4210166914","I154490720","I113428412" #Centrale Lyon, Univ Lyon, ENTPE, ENISE, ENS Lyon
         , "I48430043" ,"I4210123735",'I100532134', "I188626449", "I874609350","I4210104684" #INSA Lyon, Sciences Po Lyon, Lyon I, Lyon II, Lyon III, VetAgroSup
          #"I4387153840" "I4210155509" # Inserm Paris CNRS Rhône Alpes
        ]

#,     'isite_lorraine':#https://anr.fr/ProjetIA-15-IDEX-0004
#        ["I90183372","I4210100260", "I4210141980" ,"I1288283403", #univ Lorraine, chu nancy, inserm nancy,georgia tech nancy,
#         "I4210098696" #Institut Régional de médecine physique et de Réadaptation de Nancy
#        ]
#,     'isite_
,     "multi_idex": 
        ["I1294671590", "I4210166444","I4387153840","I4210128565",'I1326498283',"I70768539",'I4210155509', # CNRS, IRD Marseille, Inserm Paris, CEA Saclay, INRIA, Mines Paris, CNRS Rhône-Alpes
        ]
        ,'no_idex':[]
}


In [30]:
expr_idex = "CASE WHEN " + ' WHEN '.join([ 'inst_id = "' + ('" OR inst_id = "').join(item[1])   + '" THEN "' + item[0] +'"' for item in idex.items()]) + ' ELSE  "no_idex" END'
expr_idex

'CASE WHEN inst_id = "I21491767" OR inst_id = "I4210142724" OR inst_id = "I4210087487" OR inst_id = "I4387155629" THEN "idex_aix_marseille" WHEN inst_id = "I195731000" OR inst_id = "I22248866" OR inst_id = "I83860666" OR inst_id = "I201181511" OR inst_id = "I11559806" OR inst_id = "I142476485" OR inst_id = "I4210091746" OR inst_id = "I205703379" OR inst_id = "I84009706" OR inst_id = "I4210088668" OR inst_id = "I2801658355" OR inst_id = "I2801148539" OR inst_id = "I4210107720" OR inst_id = "I88467170" OR inst_id = "I4210126360" THEN "idex_paris_saclay" WHEN inst_id = "I3130945570" OR inst_id = "I4210160189" OR inst_id = "I139506611" OR inst_id = "I4210091158" THEN "idex_bordeaux" WHEN inst_id = "I2746051580" OR inst_id = "I187986737" OR inst_id = "I149758196" OR inst_id = "I29607241" OR inst_id = "I98910050" OR inst_id = "I80043" OR inst_id = "I2800004676" OR inst_id = "I56435720" OR inst_id = "I4210166784" OR inst_id = "I4210107334" OR inst_id = "I4210118986" OR inst_id = "I4210126561"

In [32]:
merged = (merged
          .withColumn('idex', func.expr(expr_idex))
         )
merged.show()

+-----------+-------------------+--------------------+-------------------+-------+--------------------+
|    inst_id|              idref|                name|               type|secteur|                idex|
+-----------+-------------------+--------------------+-------------------+-------+--------------------+
|  I51101395|          027361802|Université Paris ...|         Université| public|             no_idex|
|  I90183372|          157040569|Université de Lor...|Grand établissement| public|             no_idex|
| I277688954|          241345251|Université Paris-...|         Université| public|             no_idex|
|  I68947357|          131056549|Université de Str...|         Université| public|     idex_strasbourg|
|I4210126056|086908480,193857561|Institut national...|Autre établissement| public|idex_annulee_toul...|
|I4210103002|          252404955|Université Clermo...|         Université| public|             no_idex|
|  I24715933|          026404451|Université de La ...|         U

In [33]:
merged.write.mode('overwrite').parquet('file:\\' + save_path + 'french_inst_type_w_idex.parquet')

# Get all institutions from OpenAlex

In [75]:

recoding= {   'I118618916'  :  "I899635006",
              "I36085230"   :  "I899635006",
              "I177483745"  :  "I899635006",
              "I4210103002" :  "I198244214",
              "I4210143836" :  "I198244214",
              "I184646667"  :  "I39804081",
              "I102197404"  :  "I277688954",
              'I7171862'    :  "I2279609970",
              "I59807433"   :  "I2279609970",
              "I2800379142" :  "I4210154111",
              "I3123023596" : "I56067802",
              "I4210095130" : "I208215962" 
          #  , "I4210140029": "I197518295"
}
recoding = spark.sparkContext.broadcast(recoding)
def recoding_univ(x):
    if x in list(recoding.keys()):
        return(recoding[x])
    else:
        return x
udf_recoding_univ = func.udf(recoding_univ, StringType())
list_fused = ["I21491767","I198244214","I4210142324",
                "I899635006","I899635006","I39804081",
                "I201841394","I4210154111","I204730241","I15057530",
                "I277688954","I68947357","I2279609970","I19894307"]
list_non_fr_non_uni_pub = ['I3131573726','I4210086079',
                             "I24240610","I16465266",'I35298706',
                             "I4210134562","I185839726","I4210143169",
                             "I4210127465","I10342815",
                             "I57206974","I193291145","I4210163862","I124357947",
                             "I97565354"]

In [76]:
french_inst_type_w_idex = spark.read.parquet('file:\\' + save_path + 'french_inst_type_w_idex.parquet')

In [77]:
idex_index = {
        row['idex'] : list(set(row['inst_id'] + idex[row['idex']]))  for row in french_inst_type_w_idex.groupBy('idex').agg(func.collect_set('inst_id').alias('inst_id')).collect()
}
type_index = {
        row['type'] : row['inst_id'] for row in french_inst_type_w_idex.groupBy('type').agg(func.collect_set('inst_id').alias('inst_id')).collect()
}
secteur_index = {
        row['secteur'] : row['inst_id'] for row in french_inst_type_w_idex.groupBy('secteur').agg(func.collect_set('inst_id').alias('inst_id')).collect()
}
expr_idex  = "CASE WHEN " + ' WHEN '.join([( "inst_id IN ('" + "' , '".join(item[1])  + "')" + 
                                                     " OR arrays_overlap(parent.parent_id, array('" + "','".join(item[1]) + "'))"  +
                                                     " THEN '" + item[0] +"'") for item in idex_index.items()]) + " ELSE  'no_idex' END"
expr_type  = "CASE WHEN " + ' WHEN '.join([( "inst_id IN ('" + "' , '".join(item[1])  + "')" + 
                                                     " OR arrays_overlap(parent.parent_id, array('" + "','".join(item[1]) + "'))"  +
                                                     " THEN '" + item[0] +"'") for item in type_index.items()]) + " ELSE  'undefined' END"
expr_secteur  = "CASE WHEN " + ' WHEN '.join([( "inst_id IN ('" + "' , '".join(item[1])  + "')" + 
                                                     " OR arrays_overlap(parent.parent_id, array('" + "','".join(item[1]) + "'))"  +
                                                     " THEN '" + item[0] +"'") for item in secteur_index.items()]) + " ELSE  'undefined' END"

In [95]:
inst = (df_inst
        .filter( (func.col('geo.country_code')=="FR" ) |
                (func.array_distinct(func.col('associated_institutions.country_code')) == func.array([func.lit('FR')]))
              | (func.col('homepage_url').rlike("\.fr$|\.fr/"))
                              | ( (func.col('geo.country_code').isNull() )& (func.col('homepage_url').rlike("\.fr$|\.fr/")))
                              | ( (func.col('geo.country_code').isNull() )& (func.array_distinct(func.col('associated_institutions.country_code')) == func.array([func.lit('FR')])))
               )
        .select(func.regexp_replace(func.col('id'), 'https://openalex.org/', '').alias('inst_id'), func.col('display_name').alias('name'),
                'type', 'geo.city','associated_institutions','topic_share')

        .withColumn('topic_share', func.explode_outer(func.transform(func.col('topic_share'), lambda x: func.struct(x.field.display_name.alias('field'), x.value.alias('value')))))
        .groupBy("inst_id", "name",
                'type', 'city','associated_institutions', 'topic_share.field')
        .agg(func.sum('topic_share.value').alias('value'))
        .withColumn('max_share', func.max(func.col('value')).over(Window.partitionBy('inst_id')))
        .filter( (func.col('value')>=0.5*func.col('max_share')) | (func.col('max_share').isNull()))
        .withColumn('topic_share', func.collect_set(func.struct(func.col('field').alias('field'), func.col('value').alias('share'))).over(Window.partitionBy('inst_id')))
        .withColumn('main_topic', func.concat_ws(',', func.array_except(
            func.collect_set( func.when(func.col('value')==func.col('max_share'), func.col('field'))).over(Window.partitionBy('inst_id')), func.array([func.lit(None)]) )))
        .drop('max_share','field','value')
        .distinct()
        .withColumn('parent', func.array_except(func.transform(func.col('associated_institutions'), lambda x: func.when(x.relationship == 'parent', x)) , func.array([func.lit(None)]) ))
        .withColumn('parent', func.transform(func.col('parent'), lambda x : func.struct(func.regexp_replace(x.id, 'https://openalex.org/', '').alias('parent_id'),
                      x.display_name.alias('parent_name'),
                      x.type.alias('parent_type'))))
        .drop('associated_institutions')
        .withColumn('fused', (func.col('inst_id').isin(list_fused)).cast('int'))
        .withColumn('fused', (func.arrays_overlap(func.col('parent.parent_id'), func.array([func.lit(fused_uni) for fused_uni in list_fused]) ) ).cast('int'))
        .withColumn('uni_pub', 
                  (  ( ( func.col('type') == "education")
                    & (func.lower(func.col('name')).contains('universit'))
                    & ~(func.lower(func.col('name')).contains('catholi'))
                     & ~ (func.col('inst_id').isin(list_non_fr_non_uni_pub))
                    )
                        |
                    (
                    (func.array_contains(func.col('parent.parent_type'), 'education'))
                    & ~(func.arrays_overlap(func.col('parent.parent_id'),func.array([func.lit(fused_uni) for fused_uni in list_non_fr_non_uni_pub])))
                    & (func.array_max(func.transform(func.col('parent.parent_name'), lambda x : func.when(func.lower(x).contains('universit'), 1))) ==1)
                    & (func.array_max(func.transform(func.col('parent.parent_name'), lambda x : func.when(func.lower(x).contains('catholic'), 1)
                                                    .otherwise(0))) ==0)
                    )
                    ).cast('int')
                   )
        .fillna(0, subset = ['fused','uni_pub'])
       )
cnrs_descendants = [row[0] for row in inst.filter((func.array_contains(func.col('parent.parent_id'),'I1294671590')
                                                   | (func.col('inst_id')=="I1294671590") )).select('inst_id').collect()
                   ] + ['I1294671590', 'I57995698']

inst = (inst
        .withColumn('cnrs', ( (func.col('inst_id').isin(cnrs_descendants) )
                             | (func.arrays_overlap(func.col('parent.parent_id'), func.array( [func.lit(el) for el in cnrs_descendants] )))
                             | (func.array_max(func.transform(func.col('parent.parent_name'), lambda x: x.contains('CNRS').cast('int'))) ==1)
                            ).cast('int')
                   )
        .fillna(0, subset = ['cnrs'])
        .withColumn('parent', func.when( (func.col('parent').isNull() ) | (func.col('parent') == func.array([])), func.array(func.struct(func.col('inst_id').alias('parent_id'),
                                                                                           func.col('name').alias('parent_name'),
                                                                                           func.col('type').alias('parent_type'))))
                              .otherwise(func.col('parent'))
        
       )
             #     .withColumn('parent',func.to_json(func.col('parent')))
        
       )
inst.cache()
inst.show(truncate = False)

+-----------+-------------------------------------------------------------------------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------+----+
|inst_id    |name                                                                           |type      |city          |topic_share                                                                                                                                              |main_topic           |parent   

In [96]:
inst.count() #  5106


5106

In [97]:
inst = (inst
        .withColumn('idex', func.expr(expr_idex))
        .withColumn('type_fr', func.expr(expr_type))
        .withColumn('type_fr', func.when(func.col('type_fr')!='undefined', func.col('type_fr'))
                    .when(func.col('uni_pub')==1, "Université")
                    .when((func.col('uni_pub')==0) & (func.col('type')=="education"), "École")
                    .when((func.col('name').rlike('École')), "École")
                    .when(func.col('type')=="company", "Firme")
                    .otherwise('undefined')
                   )
        .withColumn('secteur', func.expr(expr_secteur))
        .withColumn('secteur', func.when(func.col('secteur')!='undefined', func.col('secteur'))
                    .when(func.lower(func.col('name')).rlike("de france|établissement français|public|publique"),'public')
                    .when(func.lower(func.col('name')).rlike('privé'), 'privé')
                    .when( (func.col('uni_pub')==1) | (func.col('type')=="government"), "public")
                    .when(func.col('type').isin(["company","nonprofit"]), "privé")
                    .when((func.col('type')=="facility") & 
                          (func.lower(func.col('name')).rlike('observatoire|sorbonne|maison des sciences')), "public")

                    .when((func.col('type_fr')=="École") & 
                          (func.lower(func.col('name')).rlike('national|école normale|école supérieure|académie de|lycée|school of economics|chimie paris|télécom|mines|institut universitaire|ens|purpan|institut français|collège|obs')), "public")
                    .when((func.col('type_fr')=="École") , "privé")
                    .when( (func.col('type')=="healthcare")  & ( (func.lower(func.concat_ws(',', func.col('parent.parent_name'))).rlike('niversit|militaire|armée|national'))
                                                              | (func.col('name').contains('Centre Hospitalier')) ) , "public")
                    .when( (func.col('type')=="healthcare")  &(func.lower(func.concat_ws(',', func.col('parent.parent_name'))).rlike('clinique')), "privé")

                    .when( (func.concat_ws(',', func.col('parent.parent_name')).rlike('National|Publi|Ministère'))  | (func.concat_ws(',', func.col('parent.parent_type')).rlike('government')), "public")
                    .when(func.col('name').rlike('Fondation|Association'), 'privé')
                    .otherwise('undefined')
                   )
       )
inst.show(truncate = False)

+-----------+-------------------------------------------------------------------------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------+----+---------------------+----------+---------+
|inst_id    |name                                                                           |type      |city          |topic_share                                                                                                                                   

In [ ]:
inst.write.mode('overwrite').parquet('file:\\' + save_path + 'inst_fr.parquet')

# Classify all public labs

In [42]:
inst = spark.read.parquet('file:\\' + save_path + 'inst_fr.parquet')

In [43]:
inst.filter(func.col('type') == "facility").count()

1395

In [39]:
public_labs = spark.read.option("multiline","true").json('file:\\' + scanR_path + '\\fr-esr-structures-recherche-publiques-actives.json')


In [41]:
public_labs.show()

+--------------------+------------------------+-----------------+-------------------------+---------------------------+-------------------------+--------------------------+-----------------------+-------------------------+-------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+------------------------------+--------------------------+----------------------------+--------------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             adresse|annee_d_effet_historique|annee_de_creation|code_de_nature_de_tutelle|code_de_niveau_de_structure|code_de_type_de_structure|code_de_type_de_succession|code_de_type_de_tutelle|code_domaine_scientifique|     code_panel_erc

In [46]:
public_labs.groupBy('type_de_structure').count().show()

+--------------------+-----+
|   type_de_structure|count|
+--------------------+-----+
|Fondations de coo...|    6|
|        Unité propre| 1866|
|      Equipe interne|  692|
|         Unité mixte| 1551|
|Groupements d'Int...|    7|
|Unité mixte inter...|   58|
|Collaborations in...|    1|
|Groupements d'Int...|    8|
|Structures Fédéra...|  366|
|      Société privée|    6|
|Groupements de Re...|   64|
+--------------------+-----+



In [82]:
test = df_inst.filter(func.col('geo.country_code').isNull()).filter(func.col('homepage_url').rlike("\.fr$|\.fr/") ).select('display_name','homepage_url')
test.count()


886

In [58]:
test.select('geo.country_code', func.array_distinct(func.col('associated_institutions'))).show()

+------------+---------------------------------------+
|country_code|array_distinct(associated_institutions)|
+------------+---------------------------------------+
|        NULL|                                     []|
|        NULL|                                     []|
+------------+---------------------------------------+



In [86]:
(inst
 .filter(func.col('inst_id').contains("I4387154605"))).show()

+-------+----+----+----+-----------+----------+------+-----+-------+----+
|inst_id|name|type|city|topic_share|main_topic|parent|fused|uni_pub|cnrs|
+-------+----+----+----+-----------+----------+------+-----+-------+----+
+-------+----+----+----+-----------+----------+------+-----+-------+----+



In [50]:
public_labs.filter(func.col('type_de_structure') == "Unité propre").select('libelle').show(truncate= False)

+-----------------------------------------------------------------------------------------------------------------+
|libelle                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------+
|Cerema Direction Territoriale Centre-Est                                                                         |
|CENTRE DE RECHERCHES ANGLOPHONES                                                                                 |
|Institut de Recherches Philosophiques                                                                            |
|CENTRE DE DROIT INTERNATIONAL DE NANTERRE                                                                        |
|Institut Droit Ethique Patrimoine                                                                                |
|LABORATOIRE DE SOCIOLINGUISTIQUE, D'ANTHROPOLOGIE DES PRATIQUES LANGAGI

In [34]:
spark.catalog.clearCache()
spark.stop()